# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Show runtime environment

import sys, os, torch, numpy as np

print(sys.version)
print(f'Numpy version={np.__version__}')

print(f'CUDA_VERSION={os.environ["CUDA_VERSION"]}')
print(f'CUDA_DRIVER_VERSION={os.environ["CUDA_DRIVER_VERSION"]}')
print(f'CUDNN_VERSION={os.environ["CUDNN_VERSION"]}')
      
print(f'PYTORCH_VERSION={os.environ["PYTORCH_VERSION"]}')
print(f'NVIDIA Pytorch docker image=nvcr.io/nvidia/pytorch:{os.environ["NVIDIA_PYTORCH_VERSION"]}')
print(f'GPU={torch.cuda.get_device_name(0)}')

3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) 
[GCC 7.3.0]
Numpy version=1.17.2
CUDA_VERSION=10.1.243
CUDA_DRIVER_VERSION=418.87.00
CUDNN_VERSION=7.6.4.38
PYTORCH_VERSION=1.3.0a0+24ae9b5
NVIDIA Pytorch docker image=nvcr.io/nvidia/pytorch:19.10
GPU=TITAN X (Pascal)


In [2]:
# Make results reproducible for parameter tuning

import numpy as np

def unrandomise():
    np.random.seed(42)
    torch.manual_seed(42)

    if torch.cuda.is_available():
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [3]:
unrandomise()

In [4]:
# Some constants

NUM_THREADS = 8
BATCH_SIZE = 128

In [5]:
# Imports here

from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [6]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [7]:
# Setup

import torch

do_pin_memory = torch.cuda.is_available()

In [8]:
# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(25),
    transforms.RandomResizedCrop(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

image_transforms = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [9]:
# TODO: Load the datasets with ImageFolder
train_dataset = ImageFolder(root=train_dir,
                            transform=train_transforms)

valid_dataset = ImageFolder(root=valid_dir, 
                            transform=image_transforms)

test_dataset = ImageFolder(root=test_dir, 
                           transform=image_transforms)

In [10]:
# Implement train/validation data subset for parameter tuning

DO_SUBSET = False #True
SUBSET_SIZE = 0.2

import torch.utils.data as data

def make_subset(ds, size):
    size = int(len(ds) * size)
    indices = np.arange(size)
    return data.Subset(ds, indices)

if DO_SUBSET is True:
    train_dataset = make_subset(train_dataset, SUBSET_SIZE)    
    valid_dataset = make_subset(valid_dataset, SUBSET_SIZE)    
    
print(f'Dataset sizes are train={len(train_dataset)}, validate={len(valid_dataset)}, test={len(test_dataset)}')

Dataset sizes are train=6552, validate=818, test=819


In [11]:
# TODO: Using the image datasets and the trainforms, define the dataloaders
train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE, 
                              shuffle=True, 
                              num_workers=NUM_THREADS,
                              pin_memory=do_pin_memory)

valid_dataloader = DataLoader(dataset=valid_dataset,
                              batch_size=BATCH_SIZE, 
                              shuffle=False, 
                              num_workers=NUM_THREADS,
                              pin_memory=do_pin_memory)

test_dataloader = DataLoader(dataset=test_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=False,
                             num_workers=NUM_THREADS,
                             pin_memory=do_pin_memory)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [12]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [13]:
# TODO: Build and train your network

from torchvision.models import resnext101_32x8d
from torch import nn
from collections import OrderedDict
from torch import optim

In [14]:
# Setup GPU if available

import torch

compute_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [15]:
# Freeze parameters so we don't backprop through them

def freeze_unfreeze_layer(layer, unfreeze=False):
    for name, param in layer.named_parameters():
#         print(f'{"Freeze" if unfreeze is False else "Unfreeze"} parameter: {name}')
        param.requires_grad = unfreeze

In [16]:
def train(train_dl, model, criterion, optimiser, scheduler=None, compute_device=torch.device('cpu')):
    
    model = model.train()
    running_loss = 0
    
    for images, labels in train_dl:
        images, labels = images.to(device=compute_device), labels.to(device=compute_device)

        optimiser.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()     
        optimiser.step()        
        running_loss += loss.item()
    else:
        return running_loss / len(train_dl)

In [17]:
def validate(valid_dl, model, criterion, compute_device=torch.device('cpu')):
    
    model = model.eval()
    running_loss = 0
    running_accuracy = 0

    for images, labels in valid_dl:
        images, labels = images.to(device=compute_device), labels.to(device=compute_device)

        with torch.no_grad():
            outputs = model(images)
            raw_proba = torch.exp(outputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            pred_labels = torch.argmax(raw_proba, dim=1)
            accuracy = torch.sum(pred_labels == labels) / float(pred_labels.shape[0])
            running_accuracy += accuracy
    else:
        return running_accuracy / len(valid_dl), running_loss / len(valid_dl)

In [ ]:
def save_

>> Iterate here

In [18]:
unrandomise()

In [19]:
model = resnext101_32x8d(pretrained=True)
# print(model)

In [20]:
freeze_unfreeze_layer(model)

In [21]:
# Setup classifier and optimiser parameters

num_categories = len(cat_to_name)
num_final_layers = 2048
dropout = 0.25
learning_rate = 0.001

In [22]:
new_classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(num_final_layers, 1024)),
    ('bn1', nn.BatchNorm1d(1024)),
    ('relu1', nn.PReLU()),
    ('dropout1', nn.Dropout(p=dropout)),
    ('fc2', nn.Linear(1024, 512)),
    ('bn2', nn.BatchNorm1d(512)),
    ('relu2', nn.PReLU()),
    ('dropout2', nn.Dropout(p=dropout)),
    ('fc3', nn.Linear(512, num_categories)),
    ('softmax', nn.LogSoftmax(dim=1))
]))

In [23]:
# Initialise weights in new classifier

import torch.nn.init as init

def init_weights(module):
    for name, param in module.named_parameters():
#         print(f'Initialising weights at: {name}')
        if len(param.shape) < 2:
            init.uniform_(param.data, -0.08, 0.08)
        else:
            init.kaiming_uniform_(param)

In [24]:
init_weights(new_classifier)

In [25]:
model.fc = new_classifier

In [26]:
# Move model to GPU if available

model = model.to(device=compute_device)

In [27]:
criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimiser = optim.Adam(model.fc.parameters(), lr=learning_rate)

# scheduler = optim.lr_scheduler.StepLR(optimiser, step_size=3)
scheduler = None

In [28]:
NUM_EPOCHS = 14

In [29]:
# Run training epochs

best_valid_loss = float('inf')

for e in range(NUM_EPOCHS):
    train_loss = train(train_dataloader, model, 
                       criterion=criterion, 
                       optimiser=optimiser, 
                       compute_device=compute_device)
    valid_accuracy, valid_loss = validate(valid_dataloader, model, 
                                          criterion=criterion, 
                                          compute_device=compute_device)
    if scheduler is not None:
        scheduler.step()
    
    print(f'Epoch {e + 1} - training loss={train_loss:.6f}' \
          f', validation accuracy={valid_accuracy:.6f} and loss={valid_loss:.6f}')

Epoch 1 - training loss=4.178681, validation accuracy=0.346295 and loss=3.586264
Epoch 2 - training loss=2.812023, validation accuracy=0.667411 and loss=1.960964
Epoch 3 - training loss=1.431848, validation accuracy=0.868125 and loss=0.872478
Epoch 4 - training loss=0.714759, validation accuracy=0.908571 and loss=0.496283
Epoch 5 - training loss=0.475556, validation accuracy=0.924330 and loss=0.348337
Epoch 6 - training loss=0.367356, validation accuracy=0.928304 and loss=0.330302
Epoch 7 - training loss=0.301465, validation accuracy=0.942946 and loss=0.282601
Epoch 8 - training loss=0.289158, validation accuracy=0.921830 and loss=0.303547
Epoch 9 - training loss=0.254559, validation accuracy=0.934375 and loss=0.261119
Epoch 10 - training loss=0.253550, validation accuracy=0.926563 and loss=0.266254
Epoch 12 - training loss=0.243352, validation accuracy=0.944554 and loss=0.230551
Epoch 13 - training loss=0.235646, validation accuracy=0.951875 and loss=0.226150
Epoch 14 - training loss=

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set


In [ ]:
test_accuracy, test_loss = validate(test_dataloader, model, 
                                    criterion=criterion, 
                                    compute_device=compute_device)

print(f'Testing loss: {test_loss:.6f}, and test accuracy is: {test_accuracy:.6f}')

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes